In [ ]:
import sys
import xarray as xr
import random
import numpy as np
import importlib as imp
import pandas as pd
import matplotlib.pyplot as plt
#import matplotlib.colors as mcolors

import utils
import databuilder.filemethods as filemethods
from databuilder.data_generator import ClimateData
import databuilder.data_generator as data_generator
import databuilder.filemethods as filemethods

print(f"python version = {sys.version}")
print(f"numpy version = {np.__version__}")
print(f"xarray version = {xr.__version__}")

In [ ]:
imp.reload(utils)

config = utils.get_config("exp000")
seed = config["seed_list"][0]

In [ ]:
imp.reload(data_generator)

data = ClimateData(
    config["databuilder"], 
    expname = config["expname"],
    seed=seed,
    data_dir = config["data_dir"], 
    figure_dir=config["figure_dir"],
    fetch=True,
    verbose=False
)

In [ ]:
# trainda = data._create_data()
# print(trainda)

config["databuilder"]["input_mask"] #add [0] here? 

In [ ]:
config["databuilder"]["input_region"] 

In [ ]:
dtrainx = data.d_train["x"]
dtrainx_specific = dtrainx.sel(lat = 55, lon = 10, method = "nearest")
dtrainx_specific.plot()

In [ ]:
dt = train_ds["TS"]
dt = dt.expand_dims(dim={"channel": 1}, axis = -1)
dt

In [33]:
train_ds = xr.open_dataset("/Users/C830793391/BIG DATA/E3SM Data/ens1/input_vars.v2.LR.historical_0101.eam.h1.1850-1852.nc")

da = train_ds["PRECT"]
da = da.expand_dims(dim={"channel": 1}, axis = -1)

dt = train_ds["TS"]
dt = dt.expand_dims(dim={"channel": 1}, axis = -1)

da

ds = xr.concat([da, dt], dim ="channel") 
# da.sel(channel = 1)

In [37]:
ds

<xarray.DataArray 'PRECT' (time: 1095, lat: 180, lon: 360, channel: 2)> Size: 568MB
array([[[[8.72354189e-10, 2.50529495e+02],
         [8.74143369e-10, 2.50494156e+02],
         [8.75868822e-10, 2.50458740e+02],
         ...,
         [8.66614169e-10, 2.50634735e+02],
         [8.68588201e-10, 2.50599823e+02],
         [8.70502059e-10, 2.50564728e+02]],

        [[5.97882577e-10, 2.50800079e+02],
         [6.05624106e-10, 2.50749176e+02],
         [6.13347539e-10, 2.50697815e+02],
         ...,
         [5.74621351e-10, 2.50950424e+02],
         [5.82374038e-10, 2.50900681e+02],
         [5.90130222e-10, 2.50850586e+02]],

        [[5.43213641e-10, 2.51787308e+02],
         [5.59561231e-10, 2.51739487e+02],
         [5.76133141e-10, 2.51690735e+02],
         ...,
...
         ...,
         [1.09257647e-09, 2.50732224e+02],
         [1.07936060e-09, 2.50815079e+02],
         [1.06540354e-09, 2.50897217e+02]],

        [[8.42076464e-10, 2.51446106e+02],
         [8.42475423e-10, 2.51497574e+02],
         [8.42894587e-10, 2.51549500e+02],
         ...,
         [8.40884695e-10, 2.51294556e+02],
         [8.41292702e-10, 2.51344604e+02],
         [8.41686110e-10, 2.51395126e+02]],

        [[1.30652134e-09, 2.51858078e+02],
         [1.30919187e-09, 2.51886826e+02],
         [1.31178413e-09, 2.51915665e+02],
         ...,
         [1.29806310e-09, 2.51772537e+02],
         [1.30095412e-09, 2.51800903e+02],
         [1.30377464e-09, 2.51829422e+02]]]], dtype=float32)
Coordinates:
  * time     (time) object 9kB 1850-01-01 00:00:00 ... 1852-12-31 00:00:00
  * lon      (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat      (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Dimensions without coordinates: channel
Attributes:
    long_name:     Total (convective and large-scale) precipitation rate (liq...
    units:         m/s
    cell_methods:  time: mean

In [ ]:
da.shape
da[:,:,:,0].shape

In [ ]:
train_ds["TS"].values

/System/Volumes/Data/Users/C830793391/miniconda3/pkgs/libtiff-4.5.1-h313beb8_0/lib/libtiff.5.dylib


/System/Volumes/Data/Users/C830793391/miniconda3/envs/jvp1/lib/libtiff.5.dylib

Users/C830793391/miniconda3/envs/env-torch/lib/libtiff.6.dylib




In [ ]:
# import gzip, pickle

# data_savename = "/Users/eabarnes/big_data/e3sm_v2/presaved_data/exp001_train.pkl"
# with gzip.open(data_savename, "wb") as fp:
#     pickle.dump(x_train, fp)
#     pickle.dump(labels_train, fp)

#     pickle.dump(x_mean, fp)
#     pickle.dump(x_std, fp)
    
#     pickle.dump(lat, fp)
#     pickle.dump(lon, fp)

# data_savename = "/Users/eabarnes/big_data/e3sm_v2/presaved_data/exp001_val_test.pkl"
# with gzip.open(data_savename, "wb") as fp:
#     pickle.dump(x_val, fp)
#     pickle.dump(labels_val, fp)

#     pickle.dump(x_test, fp)
#     pickle.dump(labels_test, fp)
    
#     pickle.dump(lat, fp)
#     pickle.dump(lon, fp)